In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import keras_cv
import tensorflow as tf

In [ ]:
model = keras_cv.models.StableDiffusion(
    img_width=512, img_height=512, jit_compile=False
)

In [ ]:
unet = model.diffusion_model

unet.summary()

In [ ]:
layers = ["spatial_transformer", "spatial_transformer_2", "spatial_transformer_4", "spatial_transformer_6"]

In [ ]:
import tf2onnx

i = 0
# detect all layers with name starting with "spatial_transformer"
for layer in unet.layers:
    if layer.name in layers:
        inputs = [
            tf.keras.Input(shape=layer.input[0].shape.as_list()[1:], name="input_0"),
            tf.keras.Input(shape=layer.input[1].shape.as_list()[1:], name="context"),
        ]
        specs = [
            tf.TensorSpec([1]+inputs[0].shape.as_list()[1:], inputs[0].dtype, name=inputs[0].name),
            tf.TensorSpec([1]+inputs[1].shape.as_list()[1:], inputs[1].dtype, name=inputs[1].name),
        ]
        nchw = [inputs[0].name]

        outputs = layer(inputs)
        model_i = tf.keras.Model(inputs=inputs, outputs=outputs, name=layer.name)
        tf2onnx.convert.from_keras(
            model_i,
            input_signature=specs,
            inputs_as_nchw=nchw,
            opset=18,
            output_path=f'{layer.name}/model.onnx'
        )

In [ ]:
inputs = [
    tf.keras.Input(shape=unet.input[0].shape.as_list()[1:], name="latent"),
    tf.keras.Input(shape=unet.input[1].shape.as_list()[1:], name="timestep_embedding"),
    tf.keras.Input(shape=unet.input[2].shape.as_list()[1:], name="context"),
]
specs = [
    tf.TensorSpec([1]+inputs[0].shape.as_list()[1:], inputs[0].dtype, name=inputs[0].name),
    tf.TensorSpec([1]+inputs[1].shape.as_list()[1:], inputs[1].dtype, name=inputs[1].name),
    tf.TensorSpec([1]+inputs[2].shape.as_list()[1:], inputs[2].dtype, name=inputs[2].name),
]
nchw = [inputs[0].name]

tf2onnx.convert.from_keras(
    unet,
    input_signature=specs,
    inputs_as_nchw=nchw,
    opset=18,
    output_path=f'unet.onnx'
)